## 使用飞浆构建神经网络过程

- 数据处理：从本地或URL读取数据，并完成预处理操作（如数据校验、格式转换等），保证模型可读取。
- 模型设计：网络结构设计，相当于模型的假设空间，即模型能够表达的关系集合。
- 训练配置：设定模型采用的寻解算法，即优化器，并指定计算资源。
- 训练过程：循环调用训练过程，每轮都包括前向计算、损失函数（优化目标）和后向传播三个步骤。
- 模型保存：将训练好的模型保存，模型预测时调用。

In [9]:
"""导入相关的库

paddle: 飞浆主库，根目录下保留了常用API的别名，当前包括paddle.tensor, paddle.framework, paddle.device目录下的所有API。
paddle.nn: 组网相关的API, 包括Linear, 卷积Conv2D, 循环神经网络LSTM, 损失函数CrossEntropyLoss, 激活函数ReLU等。
Linear: 神经网络的全连接层函数，包含所有输入权重相加的基本神经结构。
    在本房价预测任务中，使用只有一层的神经网络（全连接层）实现线性回归模型。
paddle.nn.functional: 与paddle.nn一样，包含组网相关的API,两者包含的同名模块功能相同，运行性能也基本一致。
    差别在于paddle.nn目录下的模块均是类，每个类自带模块参数。
    paddle.nn.functional目录下的模块均是函数，需要手动传入函数计算所需要的参数。
    在实际使用中，卷积、全连接层等本身具有可学习的参数，建议使用paddle.nn;
    而激活函数、池化等操作没有可学习的参数，可以考虑使用paddle.nn.functional。
    
    
飞浆支持两种深度学习建模编写方式，更方便调试的动态图模式和性能更好并便于部署的静态图模式。
- 动态图模式（命令式编程范式）：解析式的执行方式。用户无需预先定义完整的网络结构，每写一行网络代码，即可同时获得计算结果。
- 静态图模式（声明式编程范式）：先编译后执行的方式。用户需预先定义完整的网络结构，再对网络结构进行编译优化后，才能执行获得计算结果。
飞浆默认使用动态图模式进行编码，可通过装饰器（to_static）进行动转静支持，并可保存静态模型以实现推理部署。
"""

import paddle
from paddle.nn import Linear
import paddle.nn.functional as F

import numpy as np
import os
import random

## 数据处理

In [10]:
"""数据处理不依赖框架实现"""

def load_data():
    # 从文件导入数据
    data = np.fromfile(r'../datasets/housing.data', sep=' ', dtype=np.float32)
    
    # 数据形状变换
    feature_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE','DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
    feature_num = len(feature_names)
    data = data.reshape([data.shape[0] // feature_num, feature_num])
    
    # 数据集划分
    ratio = 0.8
    offset = int(data.shape[0] * ratio)
    training_data = data[:offset]
    
    # 归一化处理
    maximums = training_data.max(axis=0)
    minimums = training_data.min(axis=0)

    # 记录数据的归一化参数，在预测时对数据做归一化
    global max_values
    global min_values
    
    max_values = maximums
    min_values = minimums
    
    for i in range(feature_num):
        data[:, i] = (data[:, i] - minimums[i]) / (maximums[i] - minimums[i])
        
    # 返回训练集和测试集
    training_data = data[:offset]
    test_data = data[offset:]
    return training_data, test_data

In [11]:
# 验证处理数据程序的正确性
training_data, test_data = load_data()
print(training_data.shape)
print(training_data[1,:])

(404, 14)
[2.35922547e-04 0.00000000e+00 2.62405723e-01 0.00000000e+00
 1.72839552e-01 5.47997713e-01 7.82698274e-01 3.48961979e-01
 4.34782617e-02 1.14822544e-01 5.53191364e-01 1.00000000e+00
 2.04470202e-01 3.68888885e-01]


## 模型设计

In [12]:
"""模型设计

模型定义的实质是定义线性回归的网络结构
飞浆建议是通过创建类的方式完成模型网络的定义，该类需要继承paddle.nn.Layer父类，并在类中定义init和forward函数。
forward函数是框架指定实现前向计算逻辑的函数，程序在调用模型实例时会自动执行，forward函数中使用的网络层需要在init函数中声明。
"""

class Regressor(paddle.nn.Layer):
    # 在类的初始化函数中声明 每一层网络的实现函数
    def __init__(self):
        # 初始化父类中的一些参数
        super(Regressor, self).__init__()
        # 定义一层全连接层，输入维度是13，输出维度是1
        self.fc = Linear(in_features=13, out_features=1)
        
    # 网络的前向计算
    def forward(self, inputs):
        x = self.fc(inputs)
        return x

## 训练配置

In [13]:
"""训练配置的过程

1.指定运行训练的机器资源
2.声明模型实例
3.加载训练和测试数据
4.设置优化算法和学习率

模型实例有两种状态：训练状态train()和预测状态eval().
训练时要执行正向传播和反向传播梯度两个过程，而预测时只需要执行正向计算，为模型指定运行状态，有两点原因：
1.部分高级算子在两个状态执行的逻辑不同。
2.从性能和存储空间的考虑，预测状态时更节省内存（无需记录反向梯度），性能更好。
"""

# 声明定义好的线性回归模型
model = Regressor()
# 开启模型训练模式
model.train()
# 加载数据
training_data, test_data = load_data()
# 指定优化算法 - 随机梯度下降法SGD
# 学习率设置为0.01
opt = paddle.optimizer.SGD(learning_rate=0.01, parameters=model.parameters())

## 模型训练

In [15]:
"""模型训练

训练过程采用两层循环嵌套方式：
1.内层循环：负责整个数据集的一次遍历，采用分批次方式（batch）。
2.外层循环：定义遍历数据集的次数，通过参数EPOCH_NUM设置。

batch的取值会影响模型训练效果：
batch过大，会增大内存消耗和计算时间，且训练效果并不会明显提升（每次参数只向梯度反方向移动一小步，因此方向没必要特别精确）。
batch过小，每个batch的样本数据没有统计意义，计算的梯度方向可能偏差较大。

每次内循环需要执行的步骤：
1.数据准备：将一个批次的数据先转换成ndarray格式，再转换成Tensor格式；
2.前向计算：将一个批次的样本数据灌入网络中，计算输出结果；
3.计算损失函数：以前向计算结果和真实房价作为输入，通过损失函数square_error_cost API计算出损失函数值（Loss）；
4.反向传播：执行梯度反向传播backward函数，即从后到前逐层计算每一层的梯度，并根据设置的优化算法更新参数（opt.step函数）；
"""

# 设置batch大小
BATCH_SIZE = 10
# 设置外层循环次数
EPOCH_NUM = 10

# 定义外层循环
for epoch_id in range(EPOCH_NUM):
    # 在每轮迭代开始之前，将训练数据随机打乱
    np.random.shuffle(training_data)
    # 将训练数据进行拆分，每个batch包含10条数据
    mini_batches = [training_data[k : k+BATCH_SIZE] for k in range(0, len(training_data), BATCH_SIZE)]
    
    # 定义内层循环
    for iter_id, mini_batch in enumerate(mini_batches):
        # 获得当前批次训练数据
        x = np.array(mini_batch[:, :-1])
        # 获得当前批次训练标签（真实房价）
        y = np.array(mini_batch[:, -1:])
        # 将ndarray转换为飞浆动态图tensor的格式
        house_features = paddle.to_tensor(x)
        prices = paddle.to_tensor(y)
        
        # 前向计算
        predicts = model(house_features)
        
        # 计算损失
        loss = F.square_error_cost(predicts, label=prices)
        avg_loss = paddle.mean(loss)
        if iter_id % 20 == 0:
            print('epoch: {}, iter: {}, loss is: {}'.format(epoch_id, iter_id, avg_loss.numpy()))
            
        # 反向传播，计算每层参数的梯度值
        avg_loss.backward()
        # 更新参数，根据设置好的学习率迭代一步
        opt.step()
        # 清空梯度变量，以备下一轮计算
        opt.clear_grad()

epoch: 0, iter: 0, loss is: [0.8566502]
epoch: 0, iter: 20, loss is: [0.10539891]
epoch: 0, iter: 40, loss is: [0.02859592]
epoch: 1, iter: 0, loss is: [0.13015077]
epoch: 1, iter: 20, loss is: [0.1435532]
epoch: 1, iter: 40, loss is: [0.1139743]
epoch: 2, iter: 0, loss is: [0.21353555]
epoch: 2, iter: 20, loss is: [0.16141763]
epoch: 2, iter: 40, loss is: [0.08830484]
epoch: 3, iter: 0, loss is: [0.05405422]
epoch: 3, iter: 20, loss is: [0.08258742]
epoch: 3, iter: 40, loss is: [0.0886056]
epoch: 4, iter: 0, loss is: [0.07846634]
epoch: 4, iter: 20, loss is: [0.11423707]
epoch: 4, iter: 40, loss is: [0.0597805]
epoch: 5, iter: 0, loss is: [0.1088708]
epoch: 5, iter: 20, loss is: [0.12269257]
epoch: 5, iter: 40, loss is: [0.07232613]
epoch: 6, iter: 0, loss is: [0.07783701]
epoch: 6, iter: 20, loss is: [0.05768992]
epoch: 6, iter: 40, loss is: [0.06892546]
epoch: 7, iter: 0, loss is: [0.05629098]
epoch: 7, iter: 20, loss is: [0.06134908]
epoch: 7, iter: 40, loss is: [0.0938561]
epoch: 

## 保存并测试模型

In [17]:
"""保存并测试模型

训练模型和使用模型往往是不同的场景：
模型训练通常使用大量的线下服务器
模型预测通常使用线上提供预测服务的服务器实现或将已完成的预测模型嵌入到手机或其他终端设备中使用
"""

# 保存模型参数
paddle.save(model.state_dict(), '../models/paddle_lr_model.pdparams')

In [27]:
"""测试模型

测试过程和在应用场景中使用模型的过程一致，主要包括三个步骤：
1.配置模型预测的机器资源（本案例默认使用本机，因此无需写代码指定）；
2.将训练好的模型参数加载到模型实例中；
3.将待预测的样本特征输入到模型中，打印输出的预测结果。
"""

# 从测试数据集中抽取一条样本作为测试样本
def load_one_example():
    # 随机选择一条数据作为测试样本
    idx = np.random.randint(0, test_data.shape[0])
    one_data, label = test_data[idx, :-1], test_data[idx, -1]
    
    # 转换数据形状为[1, 13]
    one_data = one_data.reshape([1, -1])
    
    return one_data, label

In [28]:
# 加载模型参数
model_dict = paddle.load('../models/paddle_lr_model.pdparams')
# 将参数加载到模型
model.load_dict(model_dict)
# 将模型调整为校验状态
model.eval()

# 加载测试样本
one_data, label = load_one_example()
# 将数据转为动态图的variable格式
one_data = paddle.to_tensor(one_data)
# 预测结果
predict = model(one_data)

# 对结果做反归一化处理
predict = predict * (max_values[-1] - min_values[-1]) + min_values[-1]
# 对label数据做反归一化处理
label = label * (max_values[-1] - min_values[-1]) + min_values[-1]


print('预测结果为：{}, 真实值为：{}'.format(predict.numpy(), label))

预测结果为：[[17.519024]], 真实值为：16.799999237060547
